**wrangling** =
+ gathering
+ extracting
+ cleaning 
+ storing 
<br><br>
We gotta test assumptions about:
- values
- data types
- shape
<br><br>
..identify errors or outliers <br>
..and find missing values

Tabular data tools: Microsoft Office, Apache OpenOfficem Google Spreadsheets<br>
- row --> item
- column --> field
- cell --> value

In [ ]:
DATAFILE = "beatles-discography.csv"

In [ ]:
def parse_file(datafile):
    data = []
    with open(datafile, "rt") as f:
        header = f.readline().split(",")
        for line in f:
            counter = 0
            while counter < 3:
                fields = line.split(",")
                entry = {}

                for i, value in enumerate(fields):

                    entry[header[i].strip()] = value.strip()

                data.append(entry)
                counter += 1
           
    return data

In [ ]:
parse_file(DATAFILE)

In [ ]:
import csv
def parse_csv(datafile):
    data = []
    n = 0
    with open(datafile, 'rt') as sd:
        r = csv.DictReader(sd)   # csv.DictReader ssumes the first item is a header row
        for  line in r:
            data.append(line)
    return data

In [ ]:
import pprint 
d = parse_csv(DATAFILE)
pprint.pprint(d)

In [ ]:
import xlrd # xlwt creates Excel files
def parse_file2(datafile):
    
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    
    data = [[sheet.cell_value(r, col)               # Read into python list
                for col in range(sheet.ncols)]
                    for r in range(sheet.nrows)]
    
    print("\nList Comprehension")
    print("data[3][2]:",)
    print(data[3][2])
    print("\nCells in a nested loop:")
    for row in range(sheet.nrows):
        for col in range(sheet.ncols):
            if row ==50:
                print(sheet.cell_value(row, col))
    return sheet

In [ ]:
sheet = parse_file2('2013_ERCOT_Hourly_Load_Data.xls')

In [ ]:
sheet.nrows

In [ ]:
sheet.cell_type(3, 2)   # 2 indicates number float - Cell types: https://xlrd.readthedocs.io/en/latest/api.html#xlrd.sheet.Cell

In [ ]:
sheet.cell_value(3, 2)

In [ ]:
sheet.col_values(3, start_rowx=1, end_rowx=4)

In [ ]:
sheet.cell_type(1, 0)  # 3 indicates data float

In [ ]:
exceltime = sheet.cell_value(1, 0)
exceltime

In [ ]:
xlrd.xldate_as_tuple(exceltime, 0)

In [7]:
import pprint #!/usr/bin/env python
"""
Your task is as follows:
- read the provided Excel file
- find and return the min, max and average values for the COAST region
- find and return the time value for the min and max entries
- the time values should be returned as Python tuples

Please see the test function for the expected return format

"""

import xlrd
from zipfile import ZipFile
datafile = "2013_ERCOT_Hourly_Load_Data.xls"


def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile), 'r') as myzip:
        myzip.extractall()


def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)

    ### example on how you can get the data
    #sheet_data = [[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(sheet.nrows)]

    data = {
            'maxtime': (0, 0, 0, 0, 0, 0),
            'maxvalue': 0,
            'mintime': (0, 0, 0, 0, 0, 0),
            'minvalue': 0,
            'avgcoast': 0
    }
    
    excel = [[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(sheet.nrows)]
    cv = sheet.col_values(1, start_rowx=1, end_rowx=None)    # Get column 1 = "Coast"
    
    
    maxval = max(cv)
    maxpos = cv.index(maxval) + 1                            # We start form row 1
    maxtime = sheet.cell_value(maxpos, 0)                    # Column 0 (time)
    
    minval = min(cv)
    minpose = cv.index(minval) + 1
    mintime = sheet.cell_value(minpose, 0)
    data['mintime'] = xlrd.xldate_as_tuple(mintime, 0)
    data['minvalue'] = minval
    data['maxtime'] = xlrd.xldate_as_tuple(mintime, 0)
    data["maxvalue"] = maxval
    
    data['avgcoast'] = sum(cv) / float(len(cv))
    
    return data

In [8]:
pprint.pprint(parse_file("2013_ERCOT_Hourly_Load_Data.xls"))

{'avgcoast': 10976.933460679751,
 'maxtime': (2013, 2, 3, 4, 0, 0),
 'maxvalue': 18779.025510000003,
 'mintime': (2013, 2, 3, 4, 0, 0),
 'minvalue': 6602.113898999982}


In [14]:
"""
To experiment with this code freely you will have to run this code locally.
Take a look at the main() function for an example of how to use the code. We
have provided example json output in the other code editor tabs for you to look
at, but you will not be able to run any queries through our UI.
"""
import json
import requests

BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL = BASE_URL + "artist/"


# query parameters are given to the requests.get function as a dictionary; this
# variable contains some starter parameters.
query_type = {  "simple": {},
                "atr": {"inc": "aliases+tags+ratings"},
                "aliases": {"inc": "aliases"},
                "releases": {"inc": "releases"}}


def query_site(url, params, uid="", fmt="json"):
    """
    This is the main function for making queries to the musicbrainz API. The
    query should return a json document.
    """
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print("requesting", r.url)

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def query_by_name(url, params, name):
    """
    This adds an artist name to the query parameters before making an API call
    to the function above.
    """
    params["query"] = "artist:" + name
    return query_site(url, params)


def pretty_print(data, indent=4):
    """
    After we get our output, we can use this function to format it to be more
    readable.
    """
    if type(data) == dict:
        print(json.dumps(data, indent=indent, sort_keys=True))
    else:
        print(data)


def main():
    """
    Below is an example investigation to help you get started in your
    exploration. Modify the function calls and indexing below to answer the
    questions on the next quiz.

    HINT: Note how the output we get from the site is a multi-level JSON
    document, so try making print statements to step through the structure one
    level at a time or copy the output to a separate output file. Experimenting
    and iteration will be key to understand the structure of the data!
    """

    # Query for information in the database about bands named Nirvana
    results = query_by_name(ARTIST_URL, query_type["simple"], "Nirvana")
    #pretty_print(results)

    # Isolate information from the 4th band returned (index 3)
    print("\nARTIST:")
    #pretty_print(results["artists"][3])

    # Query for releases from that band using the artist_id
    artist_id = results["artists"][3]["id"]
    artist_data = query_site(ARTIST_URL, query_type["releases"], artist_id)
    releases = artist_data["releases"]

    # Print information about releases from the selected band
    print("\nONE RELEASE:")
    #pretty_print(releases[0], indent=2)

    release_titles = [r["title"] for r in releases]
    print("\nALL TITLES:")
    for t in release_titles:
        print(t)

if __name__ == '__main__':
    main()

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3ANirvana&fmt=json

ARTIST:
requesting http://musicbrainz.org/ws/2/artist/c49d69dc-e008-47cf-b5ff-160fafb1fe1f?inc=releases&fmt=json

ONE RELEASE:

ALL TITLES:
Remembering Le Futur


In [40]:
q1 = query_by_name(ARTIST_URL, query_type["simple"], "First Aid Kit")
q1["artists"]

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3AFirst+Aid+Kit&fmt=json


[{'id': '373faa02-74d7-4b1d-9b47-7574ad510f8d',
  'type': 'Group',
  'type-id': 'e431f5f6-b5d2-343d-8b36-72607fffb74b',
  'score': 100,
  'name': 'First Aid Kit',
  'sort-name': 'First Aid Kit',
  'country': 'SE',
  'area': {'id': '23d10872-f5ae-3f0c-bf55-332788a16ecb',
   'type': 'Country',
   'type-id': '06dd0ae4-8c74-30bb-b43d-95dcedf961de',
   'name': 'Sweden',
   'sort-name': 'Sweden',
   'life-span': {'ended': None}},
  'begin-area': {'id': '1127ddc2-eab3-4662-8718-6adbdeea3b10',
   'type': 'City',
   'type-id': '6fd8f29a-3d0a-32fc-980d-ea697b69da78',
   'name': 'Stockholm',
   'sort-name': 'Stockholm',
   'life-span': {'ended': None}},
  'disambiguation': 'Swedish folk band',
  'life-span': {'begin': '2007', 'ended': None},
  'tags': [{'count': 0, 'name': 'remember'}]},
 {'id': 'e4466078-fb0f-4899-98a2-7c7dfeb14714',
  'type': 'Group',
  'type-id': 'e431f5f6-b5d2-343d-8b36-72607fffb74b',
  'score': 97,
  'name': 'First Aid Kit',
  'sort-name': 'First Aid Kit',
  'country': 'ES',

In [46]:
q2 = query_by_name(ARTIST_URL, query_type["simple"], "Queen")
artist_id2 = q2["artists"][0]["id"]
artist_data2 = query_site(ARTIST_URL, query_type["releases"], artist_id2)
artist_data2["begin_area"]

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3AQueen&fmt=json
requesting http://musicbrainz.org/ws/2/artist/0383dadf-2a4e-4d10-a46a-e9e041da8eb3?inc=releases&fmt=json


{'sort-name': 'London',
 'id': 'f03d09b3-39dc-4083-afd6-159e3f0d462f',
 'name': 'London',
 'disambiguation': ''}

In [59]:
q3 = query_by_name(ARTIST_URL, query_type["aliases"], "Beatles")
q3["artists"]
# artists_id3 = q3["artists"][0]["id"]
# artists_data3 = query_site(ARTIST_URL, query_type["aliases"], artists_id3)
# pretty_print(artists_data3)

requesting http://musicbrainz.org/ws/2/artist/?inc=aliases&query=artist%3ABeatles&fmt=json


[{'id': 'b10bbbfc-cf9e-42e0-be17-e2c3e1d2600d',
  'type': 'Group',
  'type-id': 'e431f5f6-b5d2-343d-8b36-72607fffb74b',
  'score': 100,
  'name': 'The Beatles',
  'sort-name': 'Beatles, The',
  'country': 'GB',
  'area': {'id': '8a754a16-0027-3a29-b6d7-2b40ea0481ed',
   'type': 'Country',
   'type-id': '06dd0ae4-8c74-30bb-b43d-95dcedf961de',
   'name': 'United Kingdom',
   'sort-name': 'United Kingdom',
   'life-span': {'ended': None}},
  'begin-area': {'id': 'c249c30e-88ab-4b2f-a745-96a25bd7afee',
   'type': 'City',
   'type-id': '6fd8f29a-3d0a-32fc-980d-ea697b69da78',
   'name': 'Liverpool',
   'sort-name': 'Liverpool',
   'life-span': {'ended': None}},
  'life-span': {'begin': '1957-03', 'end': '1970-04-10', 'ended': True},
  'aliases': [{'sort-name': 'ビートルズ（ザ）',
    'type-id': '894afba6-2816-3c24-8072-eadb66bd04bc',
    'name': 'ザ・ビートルズ',
    'locale': 'ja',
    'type': 'Artist name',
    'primary': None,
    'begin-date': None,
    'end-date': None},
   {'sort-name': 'Beatles, Les

In [55]:
q4 = query_by_name(ARTIST_URL, query_type["simple"], "Nirvana")
artist_id4 = q4["artists"][0]["id"]
artists_data4 = query_site(ARTIST_URL, query_type["releases"], artist_id4)
artists_data4["disambiguation"]

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3ANirvana&fmt=json
requesting http://musicbrainz.org/ws/2/artist/5b11f4ce-a62d-471e-81fc-a69a8278c7da?inc=releases&fmt=json


'90s US grunge band'

In [57]:
q5 = query_by_name(ARTIST_URL, query_type["simple"], "One Direction")
artist_id5 = q5["artists"][0]["id"]
artists_data5 = query_site(ARTIST_URL, query_type["releases"], artist_id5)
pretty_print(artists_data5["life-span"]["begin"])

requesting http://musicbrainz.org/ws/2/artist/?query=artist%3AOne+Direction&fmt=json
requesting http://musicbrainz.org/ws/2/artist/1a425bbd-cca4-4b2c-aeb7-71cb176c828a?inc=releases&fmt=json
2010-07
